# 要件定義生成エージェント

In [22]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "ture"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

### ペルソナ定義

In [23]:
from pydantic import BaseModel, Field

# ペルソナを表すクラス
# State（Nodeで共有するデータ構造）で定義する
class Persona(BaseModel):
    name: str = Field(..., description="ペルソナの名前")
    background: str = Field(..., description="ペルソナの持つ背景")

# ペルソナのリスト
class Personas(BaseModel):
    personas: list[Persona] = Field(
        default_factory=list, description="ペルソナのリスト"
    )

# インタビュー内容を表すクラス
class Interview(BaseModel):
    persona: Persona = Field(..., description="インタビュー対象のペルソナ")
    question: str = Field(..., description="インタビューでの質問")
    answer: str = Field(..., description="インタビューでの回答")

# インタビュー結果のリスト
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(
        default_factory=list, description="インタビューの結果リスト"
    )

# 評価の結果を表すデータモデル
class EvaluationResult(BaseModel):
    reason: str = Field(..., description="判断の理由")
    is_sufficient: bool = Field(..., description="情報が十分かどうか")

In [24]:
# 要件定義生成AIエージェントのステート
# from langgraph.graph.message import add_messages
import operator
from typing import Annotated

class InterviewState(BaseModel):
    user_request: str = Field(..., description="ユーザからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    requirements_doc: str = Field(
        default="", description="生成された要件定義"
    )
    iteration: int = Field(
        default=0, description="ペルソナ生成とインタビューの反復回数"
    )
    is_information_sufficient: bool = Field(
        default=False, description="情報が十分かどうか"
    )


### 主要コンポーネント達
処理ごとに単体での動作を確認できるようすべてクラスで実装する  
**LangGraphノードは、これらrunする関数として実装する（後述）**

* PersonaGenerator　ペルソナを生成する
* InterviewConductor　インタビューを実施する
* InformationEvaluator　収集した情報の十分性を評価する
* RequirementsDocumentGenerator　要件定義書を生成する


In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# ペルソナ生成クラス
class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        # プロンプトテンプレート
        prompt =ChatPromptTemplate.from_messages(
            [
                ("system", "あなたはユーザインタビュー用の多様なペルソナを作成する専門家です。"),
                ("human", 
                    f"以下のユーザリクエストに関するインタビュー用に、{self.k}人の多様なペルソナを生成してください。\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "各ペルソナには名前と簡単な背景を含めてください。年齢、性別、職業、技術的専門知識において多様性を確保してください。"
                ),
            ]
        )
        # ペルソナ生成のためのチェーン
        chain = prompt | self.llm
        # ペルソナを生成
        return chain.invoke({"user_request": user_request})

In [ ]:
# インタビュークラス
class InterviewConductor():
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        # 質問を生成
        questions = self._generate_questions(
            user_request=user_request, personas=personas
        )
        # 回答を生成
        answers = self._generate_answers(personas=personas, questions=questions)
        # 質問と回答の組合せからインタビューリストを作成
        interviews = self._create_interviews(personas=personas, questions=questions, answers=answers)
        # インタビュー結果を返す
        return InterviewResult(interviews=interviews)
    
    def _generate_questions(self, user_request: str, personas: list[Persona]) -> list[str]:
        # 質問生成のためのプロンプトを定義
        question_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたはユーザ要件に基づいて適切な質問を生成する専門家です。"),
                ("human",
                    "以下のペルソナに関連するユーザリクエストについて、1つの質問を生成してください。\n\n"
                    "ユーザリクエスト: {user_request}\n"
                    "ペルソナ: {persona_name} - {persona_background}\n\n"
                    "質問は具体的で、このペルソナの視点から重要な情報を引き出すように設計してください。"
                )
            ]
        )
        # 質問のためのチェイン
        question_chain = question_prompt | self.llm | StrOutputParser()
        
        # 各ペルソナに対する質問クエリを生成
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background,
            } for persona in personas
        ]
        # 質問をバッチ処理で生成
        return question_chain.batch(question_queries)
    
    def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
        # 回答生成のためのプロンプト
        answer_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは以下のペルソナとして回答しています: {persona_name} - {persona_background}"),
                ("human",  "質問: {question}")
            ]
        )
        # 回答生成のためのチェイン
        answer_chain = answer_prompt | self.llm | StrOutputParser()

        # 各ペルソナに対する回答クエリを作成
        answer_queries = [
            {  
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question
            } for persona, question in zip(personas, questions)
        ]
        # 回答をバッチ処理で生成
        return answer_chain.batch(answer_queries)
    
    def _create_interviews(self, personas: list[Persona], questions: list[str], answers: list[str]) -> list[Interview]:
        # ペルソナごとに質問と回答の組合せからインタビューオブジェクトを作成
        return [
            Interview(persona=persona, question=question, answer=answer)
            for persona, question, answer in zip(personas, questions, answers)
        ]
    


In [ ]:
# 収集した情報の評価クラス
class InformationEvaluator():
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm.with_structured_output(EvaluationResult)
    
    # ユーザリクエストとインタビュー結果をもとに情報の十分性を評価
    def run(self, user_request: str, interviews: list[Interview]) -> EvaluationResult:
        # プロンプト定義
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは包括的な要約文書を作成するための情報の十分性を評価する専門家です。"),
                ("human", 
                    "以下のユーザリクエストとインタビュー結果に基づいて、包括的な要約文書を作成するのに十分な情報が集まったかどうかを判断してください。\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "インタビュー結果: \n{interview_results}"
                )
            ]
        )
        # 情報の十分性を評価するチェーンを作成
        chain = prompt | self.llm
        # 評価結果を返す
        return chain.invoke({
            "user_request": user_request, 
            "interview_results": "\n".join(
                f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                f"質問: {i.question}\n"
                f"回答: {i.answer}\n"
                for i in interviews
            )
        })


In [29]:
# 要件定義書生成クラス
class RequirementsDocumentGenerator():
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, interviews: list[Interview]) -> str:
        # プロンプトを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは収集した情報に基づいて要件文書を作成する専門家です。"),
                ("human", 
                    "以下のユーザリクエストと複数のペルソナからのインタビュー結果に基づいて、要件文書を作成してください。\n\n"
                    "ユーザリクエスト: {user_request}\n\n"
                    "インタビュー結果:\n{interview_results}\n"
                    "要件文書には以下のセクションを含めてください:\n"
                    "1. プロジェクト概要\n"
                    "2. 主要機能\n"
                    "3. 非機能要件\n"
                    "4. 制約条件\n"
                    "5. ターゲットユーザ\n"
                    "6. 優先順位\n"
                    "7. リスクと軽減策\n\n"
                    "出力は必ず日本語でお願いします。\n\n要件文書:",
                )
            ]
        )
        # 要件定義書を生成するチェーンを作成
        chain = prompt | self.llm | StrOutputParser()

        # 要件定義書を生成
        return chain.invoke({
            "user_request": user_request,
            "interview_results": "\n".join(
                f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                f"質問: {i.question}\n"
                f"回答: {i.answer}\n"
                for i in interviews
            )
        })

### ワークフロー構築
DocumentationAgent: 各コンポーネントをつなぎ合わせて全体のワークフロー管理

In [ ]:
from typing import Any, Optional
from langgraph.graph import StateGraph, START, END

class DocumentationAgent():
    def __init__(self, llm: ChatOpenAI, k: Optional[int] = None):
        # 各種ジェネレータの初期化
        self.persona_generator = PersonaGenerator(llm=llm, k=k)
        self.interview_conductor = InterviewConductor(llm=llm)
        self.information_evaluator = InformationEvaluator(llm=llm)
        self.requirements_generator = RequirementsDocumentGenerator(llm=llm)

        # グラフの作成
        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        # グラフの初期化
        workflow = StateGraph(InterviewState)

        # ノード追加
        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("evaluate_information", self._evaluate_information)
        workflow.add_node("generate_requirements", self._generate_requirements)

        # エントリポイント, ノード間エッジ
        workflow.add_edge(START, "generate_personas")
        workflow.add_edge("generate_personas", "conduct_interviews")
        workflow.add_edge("conduct_interviews", "evaluate_information")

        # 条件付きエッジ
        workflow.add_conditional_edges(
            "evaluate_information", lambda state: not state.is_information_sufficient and state.iteration < 5,
            {True: "generate_personas", False: "generate_requirements"},
        )
        workflow.add_edge("generate_requirements", END)

        # グラフのコンパイル
        return workflow.compile()



    """
    ここから、ノードの実装
    LangGraphのノードは、Stateを受け取ってStateの更新箇所のdictを返す関数（またはRunnable）
    - StateはInterviewState
    - 各関数の中で、対応する各コンポーネントのクラスをインスタンス化しrunしてStateの更新箇所を返す
    """
    def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
        # ペルソナの生成
        new_personas: Personas = self.persona_generator.run(state.user_request)

        print(type(new_personas))
        print(type(new_personas.personas))

        return {
            "personas": new_personas.personas,
            "iteration": state.iteration + 1,
        }
    
    def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
        # インタビューの実施
        new_interviews: InterviewResult = self.interview_conductor.run(
            state.user_request, state.personas[-5:]
        )
        return {"interviews": new_interviews.interviews}
    
    def _evaluate_information(self, state: InterviewState) -> dict[str, Any]:
        # 情報の評価
        evaluation_result: EvaluationResult = self.information_evaluator.run(
            state.user_request, state.interviews
        )
        return {
            "is_information_sufficient": evaluation_result.is_sufficient,
            "evaluation_reason": evaluation_result.reason,
        }
    
    def _generate_requirements(self, state: InterviewState) -> dict[str, Any]:
        # 要件定義書の作成
        requirements_doc: str = self.requirements_generator.run(
            state.user_request, state.interviews
        )
        return {"requirements_doc": requirements_doc}
    
    def run(self, user_request: str) -> str:
        # 初期状態の設定
        initial_state = InterviewState(user_request=user_request)
        # グラフの実行
        final_state = self.graph.invoke(initial_state)
        # 最終的な要件定義書の取得
        return final_state["requirements_doc"]


## 実行

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

def main():
    request = input("作成したいアプリケーション >")

    if len(request) < 1:
        request = "料理のメニューを提案してくれるアプリを開発したい"

    print(f"要求は '{request}' です")

    # llmを初期化
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    # 要求定義書作成AIエージェントを初期化
    agent = DocumentationAgent(llm=llm, k=5)

    # エージェントを実行し最終出力を取得
    output = agent.run(user_request=request)

    print(output)



In [36]:
main()

要求は '料理のメニューを提案してくれるアプリを開発したい' です
１
1.5
<class '__main__.Personas'>
<class 'list'>
2
3
4
5
# 要件文書

## 1. プロジェクト概要
本プロジェクトは、ユーザーに対して料理のメニューを提案するアプリケーションの開発を目的としています。ユーザーの食生活をサポートし、健康的でバランスの取れた食事を手軽に楽しめるようにすることを目指します。特に、カロリーや栄養バランスを考慮しつつ、調理時間が短く、簡単に作れるレシピを提供します。

## 2. 主要機能
1. **カロリー計算機能**: 各メニューのカロリーを自動で計算し、表示します。
2. **栄養素の表示**: タンパク質、脂質、炭水化物、ビタミン、ミネラルなどの栄養素を表示します。
3. **時短レシピ**: 30分以内で作れるレシピを豊富に提供します。
4. **食材の代替案**: 手に入らない食材やアレルギーに対応した代替食材を提案します。
5. **買い物リスト機能**: レシピに基づいた買い物リストを自動生成します。
6. **食事プランニング**: 1週間分の食事を計画できる機能を提供します。
7. **パーソナライズ機能**: ユーザーの好みや健康目標に応じてレシピをカスタマイズします。
8. **限られた食材で作れるレシピ**: 冷蔵庫にある食材を入力すると、それに基づいて作れるレシピを提案します。
9. **ステップバイステップのガイド**: 写真や動画付きで手順を説明します。
10. **予算に合わせたレシピ**: コストを抑えたレシピを提案します。
11. **お気に入り機能**: 気に入ったレシピを保存できる機能を提供します。

## 3. 非機能要件
1. **ユーザーインターフェース**: シンプルで直感的なデザインを採用し、視覚的に美しいインターフェースを提供します。
2. **パフォーマンス**: アプリはスムーズに動作し、ユーザーがストレスなく使用できるようにします。
3. **セキュリティ**: ユーザーの個人情報を適切に保護します。
4. **互換性**: iOSおよびAndroidプラットフォームで動作するように設計します。

## 4. 制約条件
1. **開発期間**: 6ヶ月以内に

In [14]:
from IPython.display import Image, display

try:
    display(Image(compiled_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    # これには追加の依存関係が必要であり、オプションです
    print(e)
    pass

HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)
